In [1]:
from datetime import datetime, timezone, timedelta
startDate = datetime(2011, 1, 23, tzinfo=timezone.utc)
endDate = datetime(2012, 1, 23, 4, 30, 0, tzinfo=timezone.utc)

In [2]:
# Last TLE before January 23rd 2012
tle_line1 = '1 27540U 02048A   12020.25000000 +.00000470 +00000-0 +10000-3 0  9999'
tle_line2 = '2 27540 071.6749 301.1875 8925704 250.0416 001.6882 00.33421879007360'

Setting up Orekit

In [3]:
import orekit
orekit.initVM()
from orekit.pyhelpers import setup_orekit_curdir
setup_orekit_curdir()

In [4]:
from org.orekit.frames import *
from org.orekit.utils import *
eme2000 = FramesFactory.getEME2000()
itrf = FramesFactory.getITRF(IERSConventions.IERS_2010, False)
# Selecting frames to use for OD
eci = eme2000
ecef = itrf

from org.orekit.models.earth import ReferenceEllipsoid
wgs84Ellipsoid = ReferenceEllipsoid.getWgs84(ecef)
from org.orekit.bodies import CelestialBodyFactory
sun = CelestialBodyFactory.getSun()
sun_pv_coordinates_provider = PVCoordinatesProvider.cast_(sun)

In [5]:
from org.orekit.propagation.analytical.tle import TLE
orekitTle = TLE(tle_line1, tle_line2)

from org.orekit.attitudes import NadirPointing
nadirPointing = NadirPointing(eci, wgs84Ellipsoid)

from org.orekit.propagation.analytical.tle import SGP4
sgp4Propagator = SGP4(orekitTle, nadirPointing, 4000.0)

Propagating

In [6]:
from orekit.pyhelpers import datetime_to_absolutedate, absolutedate_to_datetime
start_date_orekit = datetime_to_absolutedate(startDate)
end_date_orekit = datetime_to_absolutedate(endDate)
current_date_orekit = start_date_orekit
dt = 600.0

import pandas as pd
integralPosition_km = pd.DataFrame(columns=['x', 'y', 'z'])
integralVelocity_km_s = pd.DataFrame(columns=['vx', 'vy', 'vz'])
sunPosition_km = pd.DataFrame(columns=['x', 'y', 'z'])
while current_date_orekit.compareTo(end_date_orekit) <= 0:
    pv_eme2000_integral = sgp4Propagator.getPVCoordinates(current_date_orekit, eme2000)
    pv_eme2000_sun = sun_pv_coordinates_provider.getPVCoordinates(current_date_orekit, eme2000)
    
    current_datetime = absolutedate_to_datetime(current_date_orekit)
    
    integralPosition_km.loc[current_datetime] = pv_eme2000_integral.getPosition().toArray()[0:3]
    integralVelocity_km_s.loc[current_datetime] = pv_eme2000_integral.getVelocity().toArray()[0:3]
    sunPosition_km.loc[current_datetime] = pv_eme2000_sun.getPosition().toArray()[0:3]
    
    current_date_orekit = current_date_orekit.shiftedBy(dt)
integralPosition_km = 1e-3 * integralPosition_km
integralVelocity_km_s = 1e-3 * integralVelocity_km_s
sunPosition_km = 1e-3 * sunPosition_km

Converting sun position to AU

In [7]:
au2km = 149597870.7
sunPosition_au = sunPosition_km / au2km

Writing INTEGRAL trajectory to file

In [8]:
integral_pv_km = pd.concat([integralPosition_km, integralVelocity_km_s], axis=1, join='inner')
integral_pv_km.to_csv('integral-pv-km-eme2000.csv')

Writing Sun position to file

In [9]:
sunPosition_au.to_csv('sun-pos-au-eme2000.csv')